### モジュール読み込み

In [79]:
from pathlib import Path
import warnings

from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_pose_result)
from mmpose.datasets import DatasetInfo

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

import numpy as np
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm

/home/koudai/anaconda3/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 定数定義

In [80]:
# For model
device = 'cuda:0'
det_config = '../demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
pose_config = '../configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/crowdpose/hrnet_w32_crowdpose_256x192.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_crowdpose_256x192-960be101_20201227.pth'

# For image
img_root = '/home/koudai/data/sample/images/'

### モデル読み込み

In [26]:
det_model = init_detector(
    det_config, det_checkpoint, device=device.lower())
# build the pose model from a config file and a checkpoint file
pose_model = init_pose_model(
    pose_config, pose_checkpoint, device=device.lower())

load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_crowdpose_256x192-960be101_20201227.pth


### データ読み込み

In [27]:
dataset = pose_model.cfg.data['test']['type']
dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
if dataset_info is None:
    warnings.warn(
        'Please set `dataset_info` in the config.'
        'Check https://github.com/open-mmlab/mmpose/pull/663 for details.',
        DeprecationWarning)
else:
    dataset_info = DatasetInfo(dataset_info)

bbox_thr = 0.3
return_heatmap = False
output_layer_names = None

In [72]:
df = pd.DataFrame(columns=["dir", "name", "head.x", "head.y", "head.score", "neck.x", "neck.y", "neck.score", "Lshoulder.x", "Lshoulder.y", "Lshoulder.score", "Rshoulder.x", "Rshoulder.y", "Rshoulder.score"])

In [78]:
imglist = sorted(list(Path(img_root).glob("**/*.jpg")))

In [90]:
for i, imgpath in enumerate(tqdm(imglist)):
    # test a single image, the resulting box is (x1, y1, x2, y2)
    mmdet_results = inference_detector(det_model, imgpath)
    # keep the person class bounding boxes.
    det_cat_id = 1
    person_results = process_mmdet_results(mmdet_results, det_cat_id)

    pose_results, returned_outputs = inference_top_down_pose_model(
            pose_model,
            imgpath,
            person_results,
            bbox_thr=bbox_thr,
            format='xyxy',
            dataset=dataset,
            dataset_info=dataset_info,
            return_heatmap=return_heatmap,
            outputs=output_layer_names)
    
    result_kp = np.round(pose_results[0]['keypoints'][:,0:2], decimals= 0)
    result_score = np.round(pose_results[0]['keypoints'][:,2:3], decimals= 2)

    df.at[i,"dir"] = imgpath.parts[-2]
    df.at[i,"name"] = imgpath.name
    df.at[i, "head.x"] = result_kp[12,0].astype(int)
    df.at[i, "head.y"] = result_kp[12,1].astype(int)
    df.at[i, "head.score"] = result_score[12][0]
    df.at[i, "neck.x"] = result_kp[13,0].astype(int)
    df.at[i, "neck.y"] = result_kp[13,1].astype(int)
    df.at[i, "neck.score"] = result_score[13][0]
    df.at[i, "Lshoulder.x"] = result_kp[0,0].astype(int)
    df.at[i, "Lshoulder.y"] = result_kp[0,1].astype(int)
    df.at[i, "Lshoulder.score"] = result_score[0][0]
    df.at[i, "Rshoulder.x"] = result_kp[1,0].astype(int)
    df.at[i, "Rshoulder.y"] = result_kp[1,1].astype(int)
    df.at[i, "Rshoulder.score"] = result_score[1][0]

100%|██████████| 4558/4558 [00:01<00:00, 2366.57it/s]


In [91]:
df

,dir,name,head.x,head.y,head.score,neck.x,neck.y,neck.score,Lshoulder.x,Lshoulder.y,Lshoulder.score,Rshoulder.x,Rshoulder.y,Rshoulder.score
0,IMG_0017,IMG_0017_000001.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81
1,IMG_0017,IMG_0017_000002.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81
2,IMG_0017,IMG_0017_000003.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81
3,IMG_0017,IMG_0017_000004.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81
4,IMG_0017,IMG_0017_000005.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4553,IMG_0026,IMG_0026_000565.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81
4554,IMG_0026,IMG_0026_000566.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81
4555,IMG_0026,IMG_0026_000567.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81
4556,IMG_0026,IMG_0026_000568.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81


keypoints": [
            "left_shoulder",
            "right_shoulder",
            "left_elbow",
            "right_elbow",
            "left_wrist",
            "right_wrist",
            "left_hip",
            "right_hip",
            "left_knee",
            "right_knee",
            "left_ankle",
            "right_ankle",
            "head",
            "neck"
            ]